### 实验0.5b-微调后模型，完成基础的多轮对话

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 确保CUDA可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_path = "../results/checkpoint-103520"
# 加载微调后模型 (SFT) 和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map=None, 
    torch_dtype=torch.bfloat16
).to(device)


d:\ana\envs\machinelearn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### token 验证

In [22]:
print(tokenizer("😎🤔🙃😐😅😒😜😤😬😊"))    # 直接对单条文本tokenize
print(tokenizer(["Love", "I love"]))    # 使用List[str]对多条文本tokenize
print(tokenizer.decode([40, 2948]))    # 解码单条数据
print(tokenizer(["Love", "I love"], padding=True))    # 对不同长度文本做padding填充。注意观察attention_mask的变化
print(tokenizer(["Love", "I love"], padding="max_length", max_length=4))    # 设置不同的padding策略（填充至设定的最大长度）
print(tokenizer(["Love", "I love natrual language processing."], padding=True, truncation=True, max_length=4))    # 截断策略
print("PAD:", tokenizer.pad_token, "EOS:", tokenizer.eos_token, "EOS id:", tokenizer.eos_token_id)    # 确认微调时的重要特殊token
print(tokenizer(["Love<|endoftext|>", "I love<|endoftext|>"], padding="max_length", max_length=4))    # 提示：这和第5行代码的含义一样吗？

{'input_ids': [144848, 144927, 145868, 145631, 144713, 145869, 145078, 146184, 145705, 144236], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [[28251], [40, 2948]], 'attention_mask': [[1], [1, 1]]}
I love
{'input_ids': [[28251, 151643], [40, 2948]], 'attention_mask': [[1, 0], [1, 1]]}
{'input_ids': [[28251, 151643, 151643, 151643], [40, 2948, 151643, 151643]], 'attention_mask': [[1, 0, 0, 0], [1, 1, 0, 0]]}
{'input_ids': [[28251, 151643, 151643, 151643], [40, 2948, 17588, 81]], 'attention_mask': [[1, 0, 0, 0], [1, 1, 1, 1]]}
PAD: <|endoftext|> EOS: <|endoftext|> EOS id: 151643
{'input_ids': [[28251, 151643, 151643, 151643], [40, 2948, 151643, 151643]], 'attention_mask': [[1, 1, 0, 0], [1, 1, 1, 0]]}


In [13]:

# 基本的生成代码
inputs = tokenizer("Where is Shanghai?", return_tensors="pt").to("cuda")    # 注意：直接给模型输入tokens时需要先转换成张量，且保持运算设备位置一致
generate_ids = model.generate(**inputs, max_new_tokens=50)    # 将tokenizer的输出结果提供给模型用于生成，设置最大生成长度为50
print(generate_ids)
pred_str = tokenizer.decode(generate_ids[0, inputs["input_ids"].size(1):])
print(pred_str)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[ 9064,   374, 37047,    30, 37047,   374,   279,  6722,  3283,   315,
           279,  8853,   594,  5429,   315,  5616,   323,   374,  7407,   304,
           279, 23149,   949,   315,   279,  3146,    13,  1084,   374,   279,
          7772,  3283,   304,  5616,   323,   279,   220,    16,    18,   339,
          7772,  3283,   304,   279,  1879,    13, 37047,   374,  3881,   369,
          1181,  6481, 17646,    11]], device='cuda:0')
 Shanghai is the capital city of the People's Republic of China and is located in the eastern part of the country. It is the largest city in China and the 13th largest city in the world. Shanghai is known for its modern architecture,


#### 生成对话代码

In [17]:


prompt = "explain what is emoji"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are helpful and intelligent."},
    {"role": "user", "content": prompt}
]


text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,  # 是否立即分词
    add_generation_prompt=True  # 是否添加生成提示
)


print("Input to model:", text)


Input to model: <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are helpful and intelligent.<|im_end|>
<|im_start|>user
explain what is emoji<|im_end|>
<|im_start|>assistant



In [19]:
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)  # 转换为模型输入格式
print(input_ids.shape)
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,  # 增加重复惩罚
    do_sample=True,           # 启用采样
    eos_token_id=tokenizer.eos_token_id  # 设置结束标记
)

# print(outputs)
response = tokenizer.decode(outputs[0,input_ids.size(1):], skip_special_tokens=True)

# 后处理：截取完整的句子
def trim_to_complete_sentence(text):
    last_period = max(text.rfind("."), text.rfind("!"), text.rfind("?"))
    if last_period == -1:
        return text  # 如果没有标点符号，返回原文本
    return text[:last_period + 1]

# 确保输出完整句子
response = trim_to_complete_sentence(response)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


torch.Size([1, 33])
emoji stands for extended characters or symbols that can be used to represent text in a variety of languages including Chinese, Japanese, Korean, Thai, Vietnamese, Malayalam etc.


### 多轮对话    

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 加载模型和分词器

# 全局对话历史
conversation_history = []

# 模型支持的最大 token 长度
MAX_MODEL_LENGTH = tokenizer.model_max_length

# 聊天机器人主函数
def chatbot():
    global conversation_history
    print("欢迎使用 Qwen 聊天机器人！输入内容即可开始对话。\n输入 \\quit 结束会话，输入 \\newsession 清空对话历史并重新开始。\n")

    while True:
        # 用户输入
        user_input = input("用户: ").strip()
        
        print("用户输入:", user_input)
        # 退出会话
        if user_input == r"\quit":
            print("聊天机器人已退出，会话结束。")
            break
        
        # 开启新会话
        elif user_input == r"\newsession":
            conversation_history = []
            print("已清空对话历史，开启新的对话会话！")
            continue
        
        # 普通对话
        else:
            # 添加用户输入到对话历史
            conversation_history.append({"role": "user", "content": user_input})

            # 构造模型输入（整合你提供的逻辑）
            text = tokenizer.apply_chat_template(
                format_messages(conversation_history),
                tokenize=False,  # 不立即分词
                add_generation_prompt=True  # 添加生成提示
            )

            # 模型推理
            inputs = tokenizer(text, return_tensors="pt", padding=True).to(model.device)
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,  # 限制生成长度
                temperature=0.7,  # 调整生成的随机性
                top_k=50,  # 限制高概率单词的候选范围
                top_p=0.95,  # 核采样
                repetition_penalty=1.2,  # 惩罚重复生成
            )
            
            # 解码生成的响应
            response = tokenizer.decode(outputs[0, inputs["input_ids"].size(1):], skip_special_tokens=True).strip()
            # 后处理：截取完整的句子
            def trim_to_complete_sentence(text):
                last_period = max(text.rfind("."), text.rfind("!"), text.rfind("?"))
                if last_period == -1:
                    return text  # 如果没有标点符号，返回原文本
                return text[:last_period + 1]

            # 确保输出完整句子
            response = trim_to_complete_sentence(response)
            
            
           
            # 打印助手回复
            # 第一轮的时候还是会讲system的内容还有用户的内容全部添加到对话历史里面；太抽象了
            print(f"-------- \n{response}")

            # 添加助手响应到对话历史
            conversation_history.append({"role": "assistant", "content": response})

            
            print("对话历史:", conversation_history)
            print("--------------------")
            # 如果对话历史超出模型最大长度，则裁剪对话历史
            trim_conversation_history()




    # \return new_parts.strip()
# 格式化对话历史为 Qwen 支持的模板
def format_messages(history):
    """
    将对话历史格式化为模型支持的模板格式
    """
    formatted_messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are helpful and intelligent."}
    ]  # 初始包含系统指令
    formatted_messages.extend(history)  # 添加用户和助手的对话历史
    return formatted_messages

# 裁剪对话历史，保证长度不超过模型支持的最大长度
def trim_conversation_history():
    """
    当对话历史长度超过模型支持的最大长度时，裁剪最早的对话
    """
    global conversation_history
    
    # 使用 apply_chat_template 格式化对话历史
    formatted_history = tokenizer.apply_chat_template(
        conversation=conversation_history,
        tokenize=False,  # 返回未分词的字符串
        add_generation_prompt=False
    )
    
    # 对格式化后的字符串进行分词，获取 input_ids
    tokenized_history = tokenizer(formatted_history, return_tensors="pt", padding=True)
    input_length = tokenized_history["input_ids"].shape[1]

    # 如果长度未超出最大值，不做裁剪
    if input_length <= MAX_MODEL_LENGTH:
        return

    # 超出长度时，逐步移除最早的对话，直到满足长度限制
    while input_length > MAX_MODEL_LENGTH:
        conversation_history.pop(0)  # 移除最早的对话
        
        # 更新格式化后的对话历史并重新计算长度
        formatted_history = tokenizer.apply_chat_template(
            conversation=conversation_history,
            tokenize=False,
            add_generation_prompt=False
        )
        tokenized_history = tokenizer(formatted_history, return_tensors="pt", padding=True)
        input_length = tokenized_history["input_ids"].shape[1]

# 启动聊天机器人
if __name__ == "__main__":
    chatbot()


欢迎使用 Qwen 聊天机器人！输入内容即可开始对话。
输入 \quit 结束会话，输入 \newsession 清空对话历史并重新开始。



d:\ana\envs\machinelearn\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\ana\envs\machinelearn\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


用户输入: Tell me about emoji
-------- 
Sure! Emoji is a set of symbols used to represent emotions or feelings in text-based communication systems. They can be used for various purposes such as emoticons, emojis on social media platforms, and even within chatbots.
对话历史: [{'role': 'user', 'content': 'Tell me about emoji'}, {'role': 'assistant', 'content': 'Sure! Emoji is a set of symbols used to represent emotions or feelings in text-based communication systems. They can be used for various purposes such as emoticons, emojis on social media platforms, and even within chatbots.'}]
--------------------


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


用户输入: Tell me more!
-------- 
There are many different types of emojis that you may encounter depending on the platform where they're being displayed. For example, some common ones include smiley faces (e.g.
对话历史: [{'role': 'user', 'content': 'Tell me about emoji'}, {'role': 'assistant', 'content': 'Sure! Emoji is a set of symbols used to represent emotions or feelings in text-based communication systems. They can be used for various purposes such as emoticons, emojis on social media platforms, and even within chatbots.'}, {'role': 'user', 'content': 'Tell me more!'}, {'role': 'assistant', 'content': "There are many different types of emojis that you may encounter depending on the platform where they're being displayed. For example, some common ones include smiley faces (e.g."}]
--------------------


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


用户输入: Tell me about LLMs
-------- 
Leveraging the power of AI technology to generate human-like text is a fascinating topic. One such system that I would like you to know more about is GPT-3 (Generative Pre-trained Transformer 3).
对话历史: [{'role': 'user', 'content': 'Tell me about LLMs'}, {'role': 'assistant', 'content': 'Leveraging the power of AI technology to generate human-like text is a fascinating topic. One such system that I would like you to know more about is GPT-3 (Generative Pre-trained Transformer 3).'}]
--------------------


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


用户输入: How you trained?
-------- 
GPT-3 was developed using deep learning techniques from the field of natural language processing. It uses an advanced neural network architecture called transformer-based models which have been shown to be highly effective in generating coherent and contextually relevant responses.
对话历史: [{'role': 'user', 'content': 'Tell me about LLMs'}, {'role': 'assistant', 'content': 'Leveraging the power of AI technology to generate human-like text is a fascinating topic. One such system that I would like you to know more about is GPT-3 (Generative Pre-trained Transformer 3).'}, {'role': 'user', 'content': 'How you trained?'}, {'role': 'assistant', 'content': 'GPT-3 was developed using deep learning techniques from the field of natural language processing. It uses an advanced neural network architecture called transformer-based models which have been shown to be highly effective in generating coherent and contextually relevant responses.'}]
--------------------
用户输

NameError: name 'new_parts' is not defined